### Beautiful Soup
Escribe una función llamada scrape_coches_ocasion que acepta una URL como argumento y devuelve un DataFrame de pandas con información sobre coches de ocasión encontrados en la página web especificada. La función realizará las siguientes tareas:
- Realizará una solicitud HTTP GET a la 
- Comprobará el código de estado de la respuesta. Si el código no es 200 (OK), imprimirá un mensaje de error y retornará None.
- Extraerá los nombres de los coches, los precios rebajados, los precios originales y la información de los kilómetros recorridos de la página
- Organizará los datos extraídos en un DataFrame de pandas con las siguientes columnas:
        "coche": Nombres de los coches.
        "precio_rebajado": Precios rebajados
        "precio_original": Precios originales
        "km": Kilómetros recorridos
- Retornará el DataFrame resultante
- Una vez extraida toda la información deberéis almacenar toda la información en un DataFrame



In [42]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [43]:
url_coches = "https://www.ocasionplus.com/coches-ocasion?type%3DCAR%26sort%3DFECHAOP_DESC%26page%3D2"
coches = requests.get(url_coches)
print("La respuesta de la petición es:", coches.status_code)

La respuesta de la petición es: 200


In [44]:
coches_soup = BeautifulSoup(coches.content, 'html.parser')
#coches_soup
#print (coches_soup.prettify())

In [6]:
lista_nombre_coche = coches_soup.find_all ("h2", {"style": "overflow-x:hidden"})
lista_nombre_coche[:4]

[<h2 style="overflow-x:hidden">Peugeot 208 BlueHDi 100 Allure (100 CV)</h2>,
 <h2 style="overflow-x:hidden">Hyundai Kona 1.0 TGDi Klass 4x2 (120 CV)</h2>,
 <h2 style="overflow-x:hidden">Ford Focus 1.0 Ecoboost MHEV Trend+ (125 CV)</h2>,
 <h2 style="overflow-x:hidden">Hyundai i20 1.2 MPI (75CV) Essence LE</h2>]

In [7]:
lista_nombre_coche[0].text

'Peugeot 208 BlueHDi 100 Allure (100 CV)'

In [8]:
nombre_coches = []

for nombre in lista_nombre_coche:
    nombre_coches.append (nombre.text)
nombre_coches [:4]

['Peugeot 208 BlueHDi 100 Allure (100 CV)',
 'Hyundai Kona 1.0 TGDi Klass 4x2 (120 CV)',
 'Ford Focus 1.0 Ecoboost MHEV Trend+ (125 CV)',
 'Hyundai i20 1.2 MPI (75CV) Essence LE']

In [9]:
nombre_coches2 = [nombre.text for nombre in lista_nombre_coche]
nombre_coches2 [:4]

['Peugeot 208 BlueHDi 100 Allure (100 CV)',
 'Hyundai Kona 1.0 TGDi Klass 4x2 (120 CV)',
 'Ford Focus 1.0 Ecoboost MHEV Trend+ (125 CV)',
 'Hyundai i20 1.2 MPI (75CV) Essence LE']

## Precio descuento

In [32]:
precio_descuento = coches_soup.findAll ("span", {"class": "jsx-2689423372 price"})
precio_descuento[:4]

[<span class="jsx-2689423372 price">15.636<!-- -->€</span>,
 <span class="jsx-2689423372 price">14.536<!-- -->€</span>,
 <span class="jsx-2689423372 price">13.627<!-- -->€</span>,
 <span class="jsx-2689423372 price">10.900<!-- -->€</span>]

In [33]:
#precio descuento
lista_precio_descuento = []

for precio in precio_descuento:
    precio = precio.text
    precio_limpio = int(precio.split()[0].replace(".", "").replace("€", ""))
    lista_precio_descuento.append(precio_limpio)
    
lista_precio_descuento [:4]

[15636, 14536, 13627, 10900]

## Precio original

In [34]:
precio_original = coches_soup.find_all ("span", {"class": "jsx-2870186023 priceLast"})
precio_original[:4]

[<span class="jsx-2870186023 priceLast">17.200<!-- -->€</span>,
 <span class="jsx-2870186023 priceLast">15.990<!-- -->€</span>,
 <span class="jsx-2870186023 priceLast">14.990<!-- -->€</span>,
 <span class="jsx-2870186023 priceLast">11.990<!-- -->€</span>]

In [36]:
#precio descuento
lista_precio_original = []

for precio in precio_original:
    precio = precio.text
    precio_limpio = int(precio.split()[0].replace(".", "").replace("€", ""))
    lista_precio_original.append(precio_limpio)
    
lista_precio_original [:4]

[17200, 15990, 14990, 11990]

## Info kms

In [47]:
kms = coches_soup.find_all ("spa", {"class": "jsx-3960002075 icon"}) #no funcciona
kms [0:4]

[]

In [ ]:
def scrape_coches (url):
    #1. Llamar a al sitio web y el código
    url = "https://www.ocasionplus.com/coches-ocasion?type%3DCAR%26sort%3DFECHAOP_DESC%26page%3D2"
    coches = requests.get(url)
    print("La respuesta de la petición es:", coches.status_code)

    #2. Definir el Beautiful Soup
    coches_beautiful = BeautifulSoup(coches.content, 'html.parser')
    